# 说明：

这里所使用的数据增强和前面所用到完全一样，就是对输入的数据进行一定的数学变换，将可提取的特征增多。

缺点：既然“每轮”输入的数据不一样了，那就不能用“进阶”中的方法 —— 让卷积基只提取特征一次。因为现在每轮的图都不一样了，也就是每轮训练都会有新的特征要提取。—— 所以卷积基不能只用一次，要放入模型之中（权重参数还是定死的，因为卷积基还是只负责特征的提取）。—— 这样就和26中“基础”部分的程序基本一样了。

缺点：训练时间会很长！因为大型网络的卷积基要参与计算了！

#  数据处理部分：加入数据增强

In [2]:
import keras
from keras import layers
import numpy as np
import os
import shutil    # os和shutil用来处理文件

Using TensorFlow backend.


In [3]:
# 原始数据太多了，现在专门创建一个文件夹来存储一部分要用的训练集 + 测试集
base_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/try'
train_dir = os.path.join(base_dir, 'train')
train_dir_dog = os.path.join(train_dir , 'dog')
train_dir_cat = os.path.join(train_dir , 'cat')

test_dir = os.path.join(base_dir , 'test')
test_dir_dog = os.path.join(test_dir , 'dog')
test_dir_cat = os.path.join(test_dir , 'cat')

dc_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/train' # 原始数据所在路径

In [4]:
# 训练集猫狗各1000张，测试集猫狗各500张。
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    os.mkdir(train_dir)
    os.mkdir(train_dir_dog)
    os.mkdir(train_dir_cat)
    os.mkdir(test_dir)
    os.mkdir(test_dir_dog)
    os.mkdir(test_dir_cat)

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname) 
        d = os.path.join(train_dir_cat, fname)
        shutil.copyfile(s, d)   #  把s拷贝到d 

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_cat, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(train_dir_dog, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_dog, fname)
        shutil.copyfile(s, d)

### 加入数据增强： 

In [5]:
from keras.preprocessing.image import ImageDataGenerator

# 创建图片的迭代器：在里面加入数据增强的参数
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2, 
    horizontal_flip = True
)

# 注意：测试数据不能增强，因为增强了就不是原图了！
test_datagen = ImageDataGenerator(1/255)

In [6]:
# 正式创建图片的生成器：train
train_generator = train_datagen.flow_from_directory( train_dir,                  # 待读取文件的目录
                                                     target_size = (200,200),    # 图片的统一大小 
                                                     batch_size = 20,            # 每次读入20张
                                                     class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

# 正式创建图片的生成器：test
test_generator = test_datagen.flow_from_directory( test_dir,                  # 待读取文件的目录
                                                    target_size = (200,200),    # 图片的统一大小 
                                                    batch_size = 20,            # 每次读入20张
                                                    class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# 网络搭建：和26中完全一样

In [ ]:
# 导入keras内置的VGG16网络：
# weights参数：使用训练imagenet后得到的权重参数；若不设置该参数，则默认只使用VGG16的“层结构”
# include_top参数：是否引入VGG16自带的分类器
conv_base = keras.applications.VGG16( weights = 'imagenet', include_top = False)
conv_base.trainable = False

In [ ]:
from keras import layers

# 添加全连接层：
model = keras.Sequential()
model.add( conv_base )  # 把卷积基先加入：一个网络可以直接当作另外一个模型中的某些层，非常方便！
model.add( layers.GlobalAveragePooling2D() )  # 与layers.Flatten()功能一样，它的效率更高（这两年才出的）
model.add( layers.Dense(512, activation='relu') )
model.add( layers.Dense(1, activation='relu') )

In [ ]:
model.compile( optimizer='adam',
               loss = 'binary_crossentropy',
               metrics=['acc']
)

In [ ]:
# 此时输入数据是“generator生成器”，所以用model.fit_generator
history = model.fit_generator( 
     # 训练数据：
     train_generator,
     epochs = 2,
     steps_per_epoch = 100,  # 有2000张图，每次录入20张，因此100步可以走完一个epoch！不设置的话，生成器会一直循环生成
     # 测试数据：
     validation_data = test_generator, 
     validation_steps = 50   # 有1000张图，每次录入20张，故50步即可走完一个epoch！
)

# 总结

当使用Xception这样的大网络时，使用“数据增强”就很不好！因为它导致所以的卷积层都要参与到“重复卷积计算”当中！这是非常非常耗时的
（卷积层本来就十分耗时，现在卷积基又是这么大！）。

因此：我偏向于使用29种的那套操作流程：
- 使用Xception预训练网络：特征提取卷积基只用一次，训练Dense层的参数；
- 卷积基顶部几层卷积层解冻，进行微调。—— 至此，肯定比直接用预训练网络后，比自己搭建的CNN网格更好！
- 若这样的结果都不满足：保存上一步微调后的整个卷积基的权重参数，对训练数据进行数据增强，然后重回第一步但特征不提取！

说明：网络结果不好，最大的根源是训练数据不够！